
<h1 align=center><font size = 5>Segmenting and Clustering Addresses in Calgary City</font></h1>


Project is done by Maged Own

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

! pip install folium==0.5.0
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 112kB 23.9MB/s eta 0:00:01
Libraries imported.


<a id='item1'></a>


## 1. Download and Explore Dataset


In [95]:
!wget -q -O 'Calgary_data.json' https://data.calgary.ca/resource/ejve-em7f.json
print('Data downloaded!')

Data downloaded!


#### Load and explore the data


Next, let's load the data.


In [96]:
with open('Calgary_data.json') as json_data:
    Calgary_data = json.load(json_data)

Let's take a quick look at the data.


In [5]:
#Calgary_data

Let's convert the data to a data frame

In [129]:
neighborhoods_data = pd.DataFrame(Calgary_data)

Let's take a look at the first item in this list.


In [130]:
neighborhoods_data.drop(['roll_number', 'comm_code','property_type','assessment_class','unique_key','sub_property_use','land_use_designation','nr_assessed_value','location'], axis=1, inplace = True)
neighborhoods_data.head()

,roll_year,comm_name,year_of_construction,assessment_class_description,address,re_assessed_value,assessed_value,land_size_sf,land_size_ac,land_size_sm,latitude,longitude
0,2021,RESIDUAL WARD 9 - SUB AREA 9O,1981,Residential,105 1101 84 ST NE,39500,39500,NaN,NaN,NaN,NaN,NaN
1,2021,PANORAMA HILLS,NaN,Residential,100V 175 PANATELLA HL NW,10000,10000,151,0,14,NaN,NaN
2,2021,RESIDUAL WARD 9 - SUB AREA 9O,1999,Residential,805 1101 84 ST NE,125000,125000,NaN,NaN,NaN,NaN,NaN
3,2021,MCKENZIE TOWNE,NaN,Residential,48V 115 PRESTWICK VI SE,10000,10000,149,0,13.8,NaN,NaN
4,2021,DOWNTOWN EAST VILLAGE,2015,Residential,292V 650 5 ST SE,34000,34000,0,0,0,NaN,NaN


Let's explore the data 

In [131]:
neighborhoods_data.dropna(axis=0, subset=['latitude', 'longitude'], inplace = True)
neighborhoods_data.reset_index(inplace = True)

let's check the unique values in the dataset for number of communities and the properity types

In [133]:
print('The dataframe has {} communities and {} properity types.'.format(
        len(neighborhoods_data['comm_name'].unique()),
        len(neighborhoods_data['assessment_class_description'].unique())
    )
)
#neighborhoods_data.head()
neighborhoods_data["assessed_value"] = neighborhoods_data['assessed_value'].astype('int')
neighborhoods_data["latitude"] = neighborhoods_data['latitude'].astype('float')
neighborhoods_data["longitude"] = neighborhoods_data['longitude'].astype('float')
neighborhoods_data.dtypes


The dataframe has 2 communities and 2 properity types.


index                             int64
roll_year                        object
comm_name                        object
year_of_construction             object
assessment_class_description     object
address                          object
re_assessed_value                object
assessed_value                    int64
land_size_sf                     object
land_size_ac                     object
land_size_sm                     object
latitude                        float64
longitude                       float64
dtype: object

#### Use geopy library to get the latitude and longitude values of Calgary City.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.


In [124]:
address = 'Calgary City, AB'

geolocator = Nominatim(user_agent="C_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Calgary City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Calgary City are 51.0534234, -114.0625892.


#### Create a map of Calgary with neighborhoods superimposed on top.


In [135]:
map_calgary = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods_data['latitude'], neighborhoods_data['longitude'], neighborhoods_data['comm_name'], neighborhoods_data['address']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_calgary)  
    
map_calgary

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version


In [136]:
CLIENT_ID = 'MGICIYLWVCZ0L45PW42XQV1XLRRFQFIHBRBEJND1DGCVK32G' # your Foursquare ID
CLIENT_SECRET = 'AMVLK1LEOWZBCQT2431IC20UCRT1AKC5WMNF5PV3G12OMGGY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MGICIYLWVCZ0L45PW42XQV1XLRRFQFIHBRBEJND1DGCVK32G
CLIENT_SECRET:AMVLK1LEOWZBCQT2431IC20UCRT1AKC5WMNF5PV3G12OMGGY


#### Let's explore the first neighborhood in our dataframe.


Get the neighborhood's name.


In [137]:
neighborhoods_data.loc[0, 'comm_name']

'CALGARY INTERNATIONAL AIRPORT'

Get the neighborhood's latitude and longitude values.


In [139]:
neighborhood_latitude = neighborhoods_data.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods_data.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods_data.loc[0, 'comm_name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of CALGARY INTERNATIONAL AIRPORT are 51.1236116266249, -114.011796685429.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.


First, let's create the GET request URL. Name your URL **url**.


In [140]:
# type your answer here
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=MGICIYLWVCZ0L45PW42XQV1XLRRFQFIHBRBEJND1DGCVK32G&client_secret=AMVLK1LEOWZBCQT2431IC20UCRT1AKC5WMNF5PV3G12OMGGY&v=20180605&ll=51.1236116266249,-114.011796685429&radius=500&limit=100'

Send the GET request and examine the resutls


In [143]:
results = requests.get(url).json()
#results

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [144]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [145]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Starbucks,Coffee Shop,51.121778,-114.008599
1,WestJet Hangar,Airport Terminal,51.121173,-114.008189
2,Jugo Juice,Juice Bar,51.121503,-114.007615
3,Sunterra,Snack Place,51.121291,-114.007587
4,Westjet Gym,Gym,51.121112,-114.007270


And how many venues were returned by Foursquare?


In [146]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


<a id='item2'></a>


## 2. Explore Neighborhoods in Calgary


#### Let's create a function to repeat the same process to all the neighborhoods in Calgary


In [147]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called Calgary venues.


In [148]:
# type your answer here
Calgary_venues = getNearbyVenues(names=neighborhoods_data['comm_name'],
                                   latitudes=neighborhoods_data['latitude'],
                                   longitudes=neighborhoods_data['longitude']
                                  )

CALGARY INTERNATIONAL AIRPORT
CALGARY INTERNATIONAL AIRPORT
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADDLE RIDGE
SADD

#### Let's check the size of the resulting dataframe


In [149]:
print(Calgary_venues.shape)
Calgary_venues.head()

(1971, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,CALGARY INTERNATIONAL AIRPORT,51.123612,-114.011797,Starbucks,51.121778,-114.008599,Coffee Shop
1,CALGARY INTERNATIONAL AIRPORT,51.123612,-114.011797,WestJet Hangar,51.121173,-114.008189,Airport Terminal
2,CALGARY INTERNATIONAL AIRPORT,51.123612,-114.011797,Jugo Juice,51.121503,-114.007615,Juice Bar
3,CALGARY INTERNATIONAL AIRPORT,51.123612,-114.011797,Sunterra,51.121291,-114.007587,Snack Place
4,CALGARY INTERNATIONAL AIRPORT,51.123612,-114.011797,Westjet Gym,51.121112,-114.007270,Gym


Let's check how many venues were returned for each neighborhood


In [150]:
Calgary_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
CALGARY INTERNATIONAL AIRPORT,5,5,5,5,5,5
SADDLE RIDGE,1966,1966,1966,1966,1966,1966


#### Let's find out how many unique categories can be curated from all the returned venues


In [151]:
print('There are {} uniques categories.'.format(len(Calgary_venues['Venue Category'].unique())))

There are 20 uniques categories.


<a id='item3'></a>


## 3. Analyze Each Neighborhood


In [152]:
# one hot encoding
Calgary_onehot = pd.get_dummies(Calgary_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Calgary_onehot['Neighborhood'] = Calgary_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Calgary_onehot.columns[-1]] + list(Calgary_onehot.columns[:-1])
Calgary_onehot = Calgary_onehot[fixed_columns]

Calgary_onehot.head()

,Neighborhood,Airport Terminal,Bank,Child Care Service,Coffee Shop,Electronics Store,Fried Chicken Joint,Gym,Home Service,Juice Bar,Light Rail Station,Liquor Store,Nature Preserve,Pakistani Restaurant,Pet Store,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Snack Place,Supermarket
0,CALGARY INTERNATIONAL AIRPORT,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,CALGARY INTERNATIONAL AIRPORT,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,CALGARY INTERNATIONAL AIRPORT,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,CALGARY INTERNATIONAL AIRPORT,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,CALGARY INTERNATIONAL AIRPORT,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.


In [162]:
Calgary_onehot.shape

(1971, 21)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [163]:
Calgary_grouped = Calgary_onehot.groupby('Neighborhood').mean().reset_index()
Calgary_grouped

,Neighborhood,Airport Terminal,Bank,Child Care Service,Coffee Shop,Electronics Store,Fried Chicken Joint,Gym,Home Service,Juice Bar,Light Rail Station,Liquor Store,Nature Preserve,Pakistani Restaurant,Pet Store,Pharmacy,Pizza Place,Restaurant,Sandwich Place,Snack Place,Supermarket
0,CALGARY INTERNATIONAL AIRPORT,0.2,0.000000,0.000000,0.200000,0.000000,0.000000,0.2,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000
1,SADDLE RIDGE,0.0,0.095117,0.004578,0.110885,0.004069,0.028993,0.0,0.034588,0.0,0.152085,0.035097,0.001526,0.000509,0.050356,0.120549,0.185148,0.001526,0.134791,0.0,0.040183


#### Let's confirm the new size


In [169]:
Calgary_grouped.shape

(2, 21)

#### Let's print each neighborhood along with the top 5 most common venues


In [165]:
num_top_venues = 5

for hood in Calgary_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Calgary_grouped[Calgary_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CALGARY INTERNATIONAL AIRPORT----
              venue  freq
0  Airport Terminal   0.2
1       Snack Place   0.2
2       Coffee Shop   0.2
3               Gym   0.2
4         Juice Bar   0.2


----SADDLE RIDGE----
                venue  freq
0         Pizza Place  0.19
1  Light Rail Station  0.15
2      Sandwich Place  0.13
3            Pharmacy  0.12
4         Coffee Shop  0.11




#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [166]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [167]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Calgary_grouped['Neighborhood']

for ind in np.arange(Calgary_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Calgary_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,CALGARY INTERNATIONAL AIRPORT,Airport Terminal,Snack Place,Coffee Shop,Gym,Juice Bar,Bank,Child Care Service,Electronics Store,Fried Chicken Joint,Home Service
1,SADDLE RIDGE,Pizza Place,Light Rail Station,Sandwich Place,Pharmacy,Coffee Shop,Bank,Pet Store,Supermarket,Liquor Store,Home Service


<a id='item4'></a>


By this comparison we were able to identify most common places in the data found from Calgary. 

<a id='item5'></a>
